# Scratchpad

## IB Connect
https://interactivebrokers.github.io/tws-api/historical_bars.html#hd_what_to_show

## Python for Algorithmic Trading
https://oreil.ly/py4at


## References:
 - https://algotrading101.com/learn/interactive-brokers-python-api-native-guide/
 
 
 

In [72]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper  
from ibapi.contract import Contract

import threading
import time

class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []

    def tickPrice(self, reqId, tickType, price, attrib):
        if tickType == 2 and reqId == 1:
            print(tickType, ': The current ask price is: ', price)
    
    def historicalData(self, reqId, bar):
        #print(f'Time: {bar.date} Close: {bar.close}')
        self.data.append([bar.date, bar.open, bar.high, bar.low, bar.close])
        
def run_loop():
    app.run()

app = IBapi()
app.connect('host.docker.internal', 7496, 2)

api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()
time.sleep(0.5)



ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:cafarm
ERROR -1 2104 Market data farm connection is OK:eufarmnj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:uscrypto
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:afarm
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:ushmds.nj
ERROR -1 2106 HMDS data farm connection is OK:cashhmds
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefeu


In [73]:
app.data = []

eurusd_contract = Contract()
eurusd_contract.symbol = 'EUR'
eurusd_contract.secType = 'CASH'
eurusd_contract.exchange = 'IDEALPRO'
eurusd_contract.currency = 'USD'

#Request historical candles
app.reqHistoricalData(1, eurusd_contract, '', '2 D', '1 hour', 'BID', 0, 2, False, [])

In [97]:
app.data = []

eurzar_contract = Contract()
eurzar_contract.symbol = 'EUR'
eurzar_contract.secType = 'CASH'
eurzar_contract.exchange = 'IDEALPRO'
eurzar_contract.currency = 'ZAR'

#Request historical candles
app.reqHistoricalData(1, eurzar_contract, '', '4 W', '4 hours', 'BID', 0, 2, False, [])


In [98]:
#Working with Pandas DataFrames
import pandas as pd
import numpy as np

df = pd.DataFrame(app.data, columns=['DateTime','Open', 'High', 'Low', 'Close'])

df.drop(['Open', 'High', 'Low'], axis=1, inplace=True)
df.rename(columns={'Close':'price'}, inplace=True)
df.rename(columns={'DateTime':'date'}, inplace=True)


#df = pd.DataFrame(app.data, columns=['DateTime','Close'])
df['10SMA'] = df['price'].rolling(10).mean()
df['30SMA'] = df['price'].rolling(30).mean()

df['date'] = pd.to_datetime(df['date'],unit='s') 
df = df.set_index('date')
#df.to_csv('EURUSD_Hourly.csv')  
df.info()
df.tail()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 139 entries, 2023-03-26 21:15:00 to 2023-04-21 20:00:00
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   139 non-null    float64
 1   10SMA   130 non-null    float64
 2   30SMA   110 non-null    float64
dtypes: float64(3)
memory usage: 4.3 KB


,price,10SMA,30SMA
date,,,
2023-04-21 04:00:00,19.83030,19.835255,19.907750
2023-04-21 08:00:00,19.90255,19.835230,19.909733
2023-04-21 12:00:00,19.84260,19.825490,19.907778
2023-04-21 16:00:00,19.88360,19.821250,19.907790
2023-04-21 20:00:00,19.85980,19.820075,19.903762


In [9]:
app.disconnect()